<a href="https://colab.research.google.com/github/joardar-aditya/DeepLearning-1/blob/master/Twitter_Sentiment_analysis_logistic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install nltk
import nltk
import numpy as np

In [5]:
#import nltk tweets dataset 
from nltk.corpus import twitter_samples
from nltk.corpus import stopwords 



In [6]:
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

In [14]:
#Creating datatset for the testing and training 
test_set_pos = all_positive_tweets[4000:]
train_set_pos = all_positive_tweets[:4000]
test_set_neg = all_negative_tweets[4000:]
train_set_neg = all_negative_tweets[4000:]

test_x = test_set_pos + test_set_neg
train_x  = train_set_pos + train_set_neg 

test_y = np.append(np.ones((len(test_set_pos), 1)), np.zeros((len(test_set_neg), 1)), axis=0)
train_y = np.append(np.ones((len(train_set_pos), 1)), np.zeros((len(train_set_neg), 1)), axis=0)

In [15]:
test_x[0]

'Bro:U wan cut hair anot,ur hair long Liao bo\nMe:since ord liao,take it easy lor treat as save $ leave it longer :)\nBro:LOL Sibei xialan'

In [19]:
''' process tweet -: only stopwords, remove punctuations and for the frequency matrix 
Three important concepts -: 
   1) Tokenizing 
   2) Stemming 
   3) Lower casting '''
import re
import string 

from nltk.stem import PorterStemmer 
from nltk.tokenize import TweetTokenizer 


#Process an individual tweet 

stop_words_english = stopwords.words('english')

def process_tweet(tweet):
  tweet2 = re.sub(r'^RT[\s]+', '', tweet)
  tweet3 = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet2)
  tweet3 = re.sub(r'#', '', tweet3)
  
  tokenizer = TweetTokenizer(preserve_case =False, strip_handles = True, reduce_len = True)

  tweet_tokens = tokenizer.tokenize(tweet3)
  #Tokenizing Done
  tweet_words = []
  for words in tweet_tokens:
    if (words not in stop_words_english and words not in string.punctuation):
          tweet_words.append(words)

  #Clean Up Done
  tweet_stem = []
  stemmer = PorterStemmer()
  for words in tweet_words:
        stem_word = stemmer.stem(words)
        tweet_stem.append(stem_word)
  #Stemming Done 
  
  return tweet_stem


In [22]:
#Creating Frequency table
frequency_table = {}

for tweet in test_set_pos + train_set_pos: 
   tweet_stem = process_tweet(tweet)
   for word in tweet_stem:
     if (word, 1) in frequency_table.keys():
         frequency_table[(word,1)] += 1
     else:
         frequency_table[(word,1)] = 1

for tweet in test_set_neg + train_set_neg: 
   tweet_stem = process_tweet(tweet)
   for word in tweet_stem:
     if (word, 0) in frequency_table.keys():
         frequency_table[(word,0)] += 1
     else:
         frequency_table[(word,0)] = 1

print(frequency_table)

{('bro', 1): 24, ('u', 1): 175, ('wan', 1): 3, ('cut', 1): 3, ('hair', 1): 12, ('anot', 1): 1, ('ur', 1): 38, ('long', 1): 36, ('liao', 1): 2, ('bo', 1): 2, ('sinc', 1): 15, ('ord', 1): 1, ('take', 1): 43, ('easi', 1): 9, ('lor', 1): 1, ('treat', 1): 5, ('save', 1): 12, ('leav', 1): 12, ('longer', 1): 2, (':)', 1): 3568, ('lol', 1): 64, ('sibei', 1): 1, ('xialan', 1): 1, ('back', 1): 98, ('thnx', 1): 1, ('god', 1): 20, ("i'm", 1): 183, ('happi', 1): 211, ('thought', 1): 29, ('ear', 1): 12, ('malfunct', 1): 1, ('thank', 1): 620, ('good', 1): 238, ('clear', 1): 4, ('one', 1): 129, ('apolog', 1): 4, (':-)', 1): 692, ('stuck', 1): 4, ('centr', 1): 2, ('right', 1): 47, ('clown', 1): 1, ('joker', 1): 1, ('left', 1): 13, ('...', 1): 289, ('friday', 1): 116, ('follow', 1): 381, ('x', 1): 72, ('teenchoic', 1): 10, ('choiceinternationalartist', 1): 9, ('superjunior', 1): 9, ('fight', 1): 4, ('oppa', 1): 7, (':D', 1): 629, ('birthday', 1): 74, ('today', 1): 108, ('wish', 1): 37, ('hope', 1): 141,

In [1]:
''' PROCESSING FINISHED '''


''' Logistic Regression Implementation '''

' Logistic Regression Implementation '

Gradient Descent Explanation Tweet -: https://twitter.com/joardar7/status/1292144626795593729?s=20



In [2]:
def sigmoid(z):

  h = 1/(1 + exp(-z))

  return h 

no_of_iter = 100 #More iters , more training 

alpha = 10**(-4) #Hyperparamter, smaller the better but don't make it too small. 

def gradientdescent(x , y, alpha, theta, no_of_iter):
    
    m = x.shape[0]
    
    # x shape (m, n + 1) and theta  shape (n + 1 , 1) , z shape -: (m , 1), y shape is (m, 1)
    for i in range(no_of_iter):
      z = np.dot(x,theta) 

      h = sigmoid(z)

      ''' Cost function is the difference between the predicted value and the actual value of prediction, 
      Now the fun part -: 

      Cost Function for the logistic function is -: 

      J = -1/m * sum( y*log(h) + (1-y)*log(1 - h))

      WHy this is fun, the y is always 0 or 1 so if y = 0  and h is - then y = 0 for first term and log1 = 0 for second which makes error zero. 

      Remember it as the cost function of the logistic regression (every logistic regression)
      
      x remains unchanged 

      
      '''

      J = -1/m * (np.dot(y.transpose(), np.log(h)) + np.dot(1 - y.transpose(), np.log(1 - h)))
     
      theta = theta - alpha/m * np.dot(x.transpose(), (h - y))

    return J, theta  
  
      
# make x -: feature set  

def extract_feature(tweet, freq):

  x = np.zeros((1,3))

  ntweet = process_tweet(tweet)

  x[0,0] = 1 #(bias term)

  for word in ntweet :
      
      x[0, 1] += freq.get((word, 1), 0)

      x[0, 2] += freq.get((word, 0), 0)

  return x 











##Training begins -: 

Form the x, get the Y and give the theta (all 0, always has been); 

do gradient descent, and then get the prediction using the theta. 
